<a href="https://colab.research.google.com/github/DJCordhose/architecture/blob/master/notebooks/rl/berater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation (required for colab)

In [1]:
!pip install gym

In [0]:
!pip install -e git+https://github.com/openai/baselines#egg=MyProject

In [0]:
# do we really need this?
# # https://medium.com/@kaleajit27/reinforcement-learning-on-google-colab-9cb2e1ef51e
# !apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev -y

## Our environment, versioned by cnt

In [0]:
cnt = 0

In [0]:
import gym
from gym.utils import seeding
from gym import spaces

def state_name_to_int(state):
    state_name_map = {
        'S/Z': 0,
        'A': 1,
        'B': 2,
        'C': 3,
    }
    return state_name_map[state]

def int_to_state_name(state_as_int):
    state_map = {
        0: 'S/Z',
        1: 'A',
        2: 'B',
        3: 'C'
    }
    return state_map[state_as_int]
    
class BeraterEnv(gym.Env):
    """
    The Berater Problem

    Actions: 
    There are 3 discrete deterministic actions:
    - 0: First Direction
    - 1: Second Direction
    - 2: Third Direction / Go home
    """
    metadata = {'render.modes': ['ansi']}
    
    num_envs = 1

    def __init__(self):
        self.map = {
            'S/Z': [('A', 100), ('B', 400), ('C', 200 )],
            'A': [('B', 250), ('C', 400), ('S/Z', 100 )],
            'B': [('A', 250), ('C', 250), ('S/Z', 400 )],
            'C': [('A', 400), ('B', 250), ('S/Z', 200 )]
        }
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Discrete(1)

        self.reset()
        self.optimum = self.calculate_customers_reward()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        paths = self.map[self.state]
        destination, cost = paths[action]

        info = {"from": self.state, "to": destination}

        self.state = destination
        reward = (-cost + self.customer_reward[destination]) / self.optimum
        self.customer_visited(destination)
        done = destination == 'S/Z' and self.all_customers_visited()
        return state_name_to_int(self.state), reward, done, info

    def customer_visited(self, customer):
        self.customer_reward[customer] = 0

    def all_customers_visited(self):
        return self.calculate_customers_reward() == 0

    def calculate_customers_reward(self):
        sum = 0
        for value in self.customer_reward.values():
            sum += value
        return sum

    def reset(self):
        # print("Reset")
        reward_per_customer = 1000
        self.customer_reward = {
            'S/Z': 0,
            'A': reward_per_customer,
            'B': reward_per_customer,
            'C': reward_per_customer,
        }

        self.state = 'S/Z'
        return state_name_to_int(self.state)

    def render(self, mode='human'):
        print(self.state)
        print(self.customer_reward)

from gym.envs.registration import register
cnt += 1
id = "Berater-v{}".format(cnt)
register(
    id=id,
    entry_point=BeraterEnv
)        
env = gym.make(id)

## Try out Environment

In [5]:
# Run a demo of the environment
observation = env.reset()
for t in range(1000):
    env.render()
    action = env.action_space.sample()
    print("Action: {}".format(action))
    observation, reward, done, info = env.step(action)
    print(reward)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()

S/Z
{'S/Z': 0, 'A': 1000, 'B': 1000, 'C': 1000}
Action: 0
0.3
A
{'S/Z': 0, 'A': 0, 'B': 1000, 'C': 1000}
Action: 1
0.2
C
{'S/Z': 0, 'A': 0, 'B': 1000, 'C': 0}
Action: 0
-0.13333333333333333
A
{'S/Z': 0, 'A': 0, 'B': 1000, 'C': 0}
Action: 1
-0.13333333333333333
C
{'S/Z': 0, 'A': 0, 'B': 1000, 'C': 0}
Action: 1
0.25
B
{'S/Z': 0, 'A': 0, 'B': 0, 'C': 0}
Action: 2
-0.13333333333333333
Episode finished after 6 timesteps


## Train PPO2 using MLP Neural Network Architecture

In [9]:
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from baselines.ppo2 import ppo2

wrapped_env = DummyVecEnv([lambda: gym.make(id)])

model = ppo2.learn(network='mlp', env=wrapped_env, total_timesteps=200000)

Logging to /tmp/openai-2018-11-08-09-29-15-446014
--------------------------------------
| approxkl           | 4.6544883e-05 |
| clipfrac           | 0.0           |
| eplenmean          | nan           |
| eprewmean          | nan           |
| explained_variance | -0.11         |
| fps                | 456           |
| nupdates           | 1             |
| policy_entropy     | 1.0985682     |
| policy_loss        | -0.0005559514 |
| serial_timesteps   | 2048          |
| time_elapsed       | 4.49          |
| total_timesteps    | 2048          |
| value_loss         | 0.041274015   |
--------------------------------------
--------------------------------------
| approxkl           | 0.00025403075 |
| clipfrac           | 0.0           |
| eplenmean          | nan           |
| eprewmean          | nan           |
| explained_variance | 0.121         |
| fps                | 537           |
| nupdates           | 10            |
| policy_entropy     | 1.0521085     |
| policy_loss 

## Use trained model to do actual planning

In [10]:
observation = wrapped_env.reset()

import numpy as np 

state = np.zeros((1, 2*128))
dones = np.zeros((1))

for t in range(1000):
    wrapped_env.render()
    actions, _, state, _ = model.step(observation, S=state, M=dones)
    print("Action: {}".format(actions))
    observation, reward, done, info = wrapped_env.step(actions)
    print(reward)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
wrapped_env.close()

S/Z
{'S/Z': 0, 'A': 1000, 'B': 1000, 'C': 1000}
Action: [2]
[0.26666668]
C
{'S/Z': 0, 'A': 1000, 'B': 1000, 'C': 0}
Action: [1]
[0.25]
B
{'S/Z': 0, 'A': 1000, 'B': 0, 'C': 0}
Action: [2]
[-0.13333334]
S/Z
{'S/Z': 0, 'A': 1000, 'B': 0, 'C': 0}
Action: [0]
[0.3]
A
{'S/Z': 0, 'A': 0, 'B': 0, 'C': 0}
Action: [1]
[-0.13333334]
C
{'S/Z': 0, 'A': 0, 'B': 0, 'C': 0}
Action: [0]
[-0.13333334]
A
{'S/Z': 0, 'A': 0, 'B': 0, 'C': 0}
Action: [2]
[-0.03333334]
Episode finished after 7 timesteps
